In [1]:
import pandas as pd
from pandas import json_normalize
import json
import ast
# Convert the data into json string and then load it as a python list, dictionary for traversal 
def CustomParser(data):
    if(len(data))>0:
        data = ast.literal_eval(data)
        j1 = json.dumps(data)# converts a python object into a json string
        j2=json.loads(j1) # converts a valid string into a python dict/list
        return j2

In [12]:
#read the movies_metadata file with all columns 
df_source1 = pd.read_csv('File Loaded/movies_metadata.csv',header=[0],converters={'genres':CustomParser,
                                                                      'production_companies':CustomParser,
                                                                      'production_countries':CustomParser,
                                                                      'spoken_languages':CustomParser
                                                                        })



In [7]:
print(df_source1[pd.to_numeric(df_source1['id'],errors='coerce').isnull()]) # Get bad records

                                                   adult  \
19730                                 - Written by Ørnås   
29503   Rune Balot goes to a casino connected to the ...   
35587   Avalanche Sharks tells the story of a bikini ...   

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                  genres  \
19730  [{'name': 'Carousel Productions', 'id': 11176}...   
29503  [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...   
35587  [{'name': 'Odyssey Media', 'id': 17161}, {'nam...   

                                                homepage          id imdb_id  \
19730  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...  1997-08-20       0   
29503  [{'iso_3166_1': 'US', 'name': 'United States o...  2012-09-29       0   
35587           [

In [8]:
# using the source data frame loaded from flat file , create the movies data frame with relevant columns
df_movies = df_source1[['adult','belongs_to_collection','budget','homepage','id','imdb_id','original_language','original_title',
                        'overview','popularity','poster_path','production_countries','release_date','revenue','runtime','status',
                        'tagline','title','video','vote_average','vote_count']]
df_movies.to_csv('movies.csv', encoding='utf-8', index=False) 




In [9]:
# using the source data frame loaded from flat file , flatten the genres json data
df_genres= df_source1[['id', 'genres']]

df_genres = (pd.concat({i: json_normalize(x) for i, x in df_genres.pop('genres').items() if x is not None })
         .reset_index(level=1, drop=True)
         .join(df_genres, lsuffix='_genres', rsuffix='_movies'))


#df_genres.drop_duplicates() 

df_genres.to_csv('genres.csv', encoding='utf-8', index=False) 


In [10]:
# using the source data frame loaded from flat file , flatten the languages json data
df_languages= df_source1[['id', 'spoken_languages']]

df_languages = (pd.concat({i: json_normalize(x) for i, x in df_languages.pop('spoken_languages').items() if x is not None })
         .reset_index(level=1, drop=True)
         .join(df_languages, lsuffix='_lang', rsuffix='_movies'))

df_languages.to_csv('languages.csv', encoding='utf-8', index=False) 


In [13]:
# using the source data frame loaded from flat file , flatten the production companies json data
df_production_companies= df_source1[['id', 'production_companies']]
df_production_companies = (pd.concat({i: json_normalize(x) for i, x in df_production_companies.pop('production_companies').items() if x is not None })
         .reset_index(level=1, drop=True)
         .join(df_production_companies, lsuffix='_production_companies', rsuffix='_movies'))

df_production_companies.to_csv('production_companies.csv', encoding='utf-8', index=False) 
